<h1 style="color:white; ">
    importing libraries
</h1>
<ul style="color:white; font-size:20px;">
<li> re : for data cleaning and preprocessing</li>
<li> pandas : for data manipulation and analysis</li>
<li> torch : for building and training deep learning models</li>
<li> DataLoader  : for loading and preprocessing data</li>
<li> torch.nn : for building and training neural networks</li>
<li> datasets : for getting data from Hugging Face</li>
<li> build_vocab_from_iterator : for building vocabulary from text data</li>
<li> spacy : for tokenizing text </li>
<li> IPython.display  : for displaying  code</li>
<li> tokenize  : for tokenizing code</li>
<li> BytesIO  : for encoding code</li>
<li> torch.nn : for create decoder</li>
</ul>



In [1]:
import re
import torch
from torch import nn
import spacy
import pandas as pd
from datasets import load_dataset
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from IPython.display import display, Markdown
import tokenize
from io import BytesIO


/home/hassan/anaconda3/envs/Ai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<h1 style="color:white;">
data preprocessing
</h1>

<h2>
<a href="https://huggingface.co/datasets/nceyda/YAP470_Code_Generation_Dataset">dataset link</a>
</h2>

In [2]:

ds = load_dataset("nceyda/YAP470_Code_Generation_Dataset")


In [3]:
ds


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', '__index_level_0__'],
        num_rows: 74698
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', '__index_level_0__'],
        num_rows: 13182
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', '__index_level_0__'],
        num_rows: 9765
    })
})

In [4]:
inputs = ds["train"]["input"][:500]
outputs = ds["train"]["output"][:500]

inputs[-2],display(Markdown(outputs[-2]))



```ndiff
      # Some users have more than 1 default template.
      # This shouldn't be possible, make sure is will be just 1.
      User = apps.get_model('users', 'LilyUser')
      DefaultEmailTemplate = apps.get_model('email', 'DefaultEmailTemplate')
  
-     print('\nFixing default template for the following users:')
```

```ndiff
              # User has more than one default template.
              # Best guess would be that the user prefers the last set template to be the default.
              # So remove all except the last one.
              template_to_keep = templates.last()
              templates.exclude(id=template_to_keep.id).delete()
- 
-             print('%d:\t%s' % (user.pk, user.email))
```

('I\'d like to see a change in the code that achieves this: "Remove print statements, not usefull anymore."\nFor your reference, this is the current state of lily/messaging/email/migrations/0013_fix_multple_default_templates.py:\n\n```python\n# -*- coding: utf-8 -*-\nfrom __future__ import unicode_literals\n\nfrom django.db import models, migrations\n\n\ndef fix_multiple_default_templates(apps, schema_editor):\n    # Some users have more than 1 default template.\n    # This shouldn\'t be possible, make sure is will be just 1.\n    User = apps.get_model(\'users\', \'LilyUser\')\n    DefaultEmailTemplate = apps.get_model(\'email\', \'DefaultEmailTemplate\')\n\n    print(\'\\nFixing default template for the following users:\')\n    for user in User.objects.all():\n        templates = DefaultEmailTemplate.objects.filter(user=user.pk).order_by(\'id\')\n        if templates.count() > 1:\n            # User has more than one default template.\n            # Best guess would be that the user p

In [ ]:
def cleanText(text):
    text = re.sub(r'[^a-zA-Z0-9<=>\n\\u0600-\\u06FF\\u4e00-\\u9fff_.,:()\[\]{} \\]', '', text)
    return text.strip()


In [6]:
def cleanCode(code):
    lines = code.strip().split('\n')
    
    if not lines[0].startswith('```python'):
        lines.insert(0, '```python')
    
    if not lines[-1].endswith('```'):
        lines.append('```')
    
    return '\n'.join(lines)


In [7]:
def tokenizeCode(code):
    tokens = []
    try:
        for tok in tokenize.tokenize(BytesIO(code.encode('utf-8')).readline):
            if tok.type not in {tokenize.ENCODING, tokenize.COMMENT}:
                tokens.append(tok.string.strip())
    except:
        return []
    return tokens

nlp = spacy.load("en_core_web_sm")

def tokenizeText(text):
    doc = nlp(text)
    tokens = []
    for token in doc:
        if token.is_punct or token.is_space:
            continue
        tokens.append(token.text)
    return tokens


In [8]:

def yieldTokens(data, tokenizer):
    for text in data:
        yield tokenizer(text)


def buildVocab(data, tokenizer):
    vocab = build_vocab_from_iterator(
        yieldTokens(data, tokenizer),
        specials=['<unk>', '<pad>', '<sos>', '<eos>'],
        min_freq=2
    )
    vocab.set_default_index(vocab['<unk>'])
    return vocab


In [10]:
cleanedInputs = [cleanText(text) for text in inputs]
cleanedOutputs = [cleanCode(cleanText(text)) for text in outputs]


In [ ]:
outputs[-1]


'```python\n"""Templatetags for djangopress."""\nfrom datetime import date\nfrom collections import defaultdict\n\nfrom django import template\n\nfrom djangopress.models import Post, Category\n\n\nregister = template.Library()\n\n\n@register.inclusion_tag(\'djangopress/tags/archive_list.html\')\ndef archive_list():\n    """List post by date"""\n    posts = Post.objects.all()\n    years_dictionary = defaultdict(set)\n    for post in posts:\n        year = post.creation_date.year\n        month = post.creation_date.month\n        years_dictionary[year].add(month)\n    years = {}\n    for year, months in years_dictionary.items():\n        year = str(year)\n        years[year] = []\n        for month in months:\n            years[year].append(date(int(year), month, 1))\n    for year in years:\n        years[year].sort(reverse=True)\n    return {\'years\': years}\n\n\n@register.inclusion_tag(\'djangopress/tags/category_list.html\')\ndef category_list():\n    """List the categories in the bl

In [12]:
display(Markdown(cleanedOutputs[-1]))


```python
python
Templatetags for djangopress.
from datetime import date
from collections import defaultdict

from django import template

from djangopress.models import Post, Category


register = template.Library()


@register.inclusion_tag(djangopresstagsarchive_list.html)
def archive_list():
    List post by date
    posts = Post.objects.all()
    years_dictionary = defaultdict(set)
    for post in posts:
        year = post.creation_date.year
        month = post.creation_date.month
        years_dictionary[year].add(month)
    years = {}
    for year, months in years_dictionary.items():
        year = str(year)
        years[year] = []
        for month in months:
            years[year].append(date(int(year), month, 1))
    for year in years:
        years[year].sort(reverse=True)
    return {years: years}


@register.inclusion_tag(djangopresstagscategory_list.html)
def category_list():
    List the categories in the blog.
    categories = Category.objects.all()
    return {categories: categories}
```

In [13]:
display(Markdown(cleanedOutputs[3]))


```python
for key, value in my_dict.items():
    if my_dict.values().count(value) > 1:
        print(key, value)
```

In [14]:
vocabText = buildVocab(cleanedInputs, tokenizeText)
vocabCode = buildVocab(cleanedOutputs,tokenizeCode)


In [15]:
def dataProcess(Text, Code, seqLen=None):
    data = []
    for rawText, rawCode in zip(Text, Code):
        tokensText = [vocabText['<sos>']] + [vocabText[token] for token in tokenizeText(rawText)] + [vocabText['<eos>']]
        tokensCode = [vocabCode['<sos>']] + [vocabCode[token] for token in tokenizeCode(rawCode)] + [vocabCode['<eos>']]
        
        if seqLen is not None:
            if len(tokensText) > seqLen:
                tokensText = tokensText[:seqLen]
            else:
                tokensText += [vocabText['<pad>']] * (seqLen - len(tokensText))
            
            if len(tokensCode) > seqLen:
                tokensCode = tokensCode[:seqLen]
            else:
                tokensCode += [vocabCode['<pad>']] * (seqLen - len(tokensCode))
        
        data.append((torch.tensor(tokensText, dtype=torch.long),torch.tensor(tokensCode, dtype=torch.long)))
    return data


In [16]:
trainData = dataProcess(inputs,outputs,500)

device = torch.device('cuda' if torch.cuda.is_available() else  'cpu')
batchSize = 32
trainLoader = DataLoader(
    trainData, 
    batch_size=batchSize,
    shuffle=True
    )


<div style="color:white;">
<h1>
create decoder only from transformer model
</h1>
<h3> why we need decoder only from transformer model </h3>
<p>
The Transformer model is a popular architecture for sequence-to-sequence tasks, such as machine translation, text
generation, and question answering. It consists of an encoder and a decoder. The encoder takes in a
sequence of tokens ( words or characters) and produces a continuous representation of the input sequence. 
The decoder then takes this representation and generates the output sequence, one token at a time.
</p>
</div>

In [ ]:
class TransformerDecoderModel(nn.Module):
    def __init__(self, inputVocabSize, outputVocabSize, sequenceLen, embedSize, hiddenSize, numLayers, numHeads, dropout=0.1):
        super(TransformerDecoderModel, self).__init__()

        self.srcEmbedding = nn.Embedding(inputVocabSize, embedSize)
        
        self.tgtEmbedding = nn.Embedding(outputVocabSize, embedSize)
        self.positionalEncoding = nn.Parameter(torch.zeros(1, sequenceLen, embedSize))
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=embedSize, nhead=numHeads, dim_feedforward=hiddenSize, dropout=dropout
        )
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=numLayers)
        self.fc_out = nn.Linear(embedSize, outputVocabSize)
        
    def forward(self, tgt, src, tgtMask=None):
        
        srcEmb = self.srcEmbedding(src) + self.positionalEncoding[:, :src.size(1), :]
        srcEmb = srcEmb.permute(1, 0, 2)  # (seq_len, batch, embed_size)
        
        tgtEmb = self.tgtEmbedding(tgt) + self.positionalEncoding[:, :tgt.size(1), :]
        tgtEmb = tgtEmb.permute(1, 0, 2)  # (seq_len, batch, embed_size)
        
        output = self.decoder(tgtEmb, srcEmb, tgt_mask=tgtMask)
        output = output.permute(1, 0, 2)  # (batch, seq_len, embed_size)
        return self.fc_out(output)

inputVocabSize = len(vocabText)
outputVocabSize = len(vocabCode)
embedSize = 512
hiddenSize = 2048
numLayers = 6
numHeads = 8
dropout = 0.4



In [18]:
decoderModel = TransformerDecoderModel(inputVocabSize,outputVocabSize,500, embedSize, hiddenSize, numLayers, numHeads, dropout)
print(decoderModel)


TransformerDecoderModel(
  (srcEmbedding): Embedding(5925, 512)
  (tgtEmbedding): Embedding(3321, 512)
  (decoder): TransformerDecoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (multihead_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.4, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm3): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.4, inplace=False)
        (dropout2): Dropout(p=0

In [19]:
totalParams = sum(p.numel() for p in decoderModel.parameters())
totalParams


31917817

In [20]:
criterion = nn.CrossEntropyLoss(ignore_index=vocabCode['<pad>'])
optimizer = torch.optim.Adam(decoderModel.parameters(), lr=0.0001)

validInputs = ds["validation"]["input"][:500]
validOutputs = ds["validation"]["output"][:500]

validData = dataProcess(validInputs, validOutputs, 500)
validLoader = DataLoader(validData, batch_size=batchSize, shuffle=False)

patience = 10
bestValidLoss = float('inf')
patienceCounter = 0
bestModelWeights = None

decoderModel = decoderModel.to(device)

numEpochs = 1 
for epoch in range(numEpochs):
    decoderModel.train()
    totalLoss = 0
    
    for batch_idx, (srcSeq, tgtSeq) in enumerate(trainLoader):
        srcSeq = srcSeq.to(device)
        tgtSeq = tgtSeq.to(device)
        
        tgtInput = tgtSeq[:, :-1]
        tgtOutput = tgtSeq[:, 1:]

        tgtMask = nn.Transformer.generate_square_subsequent_mask(tgtInput.size(1)).to(device)
        
        outputs = decoderModel(tgt=tgtInput, src=srcSeq, tgtMask=tgtMask)
        
        loss = criterion(outputs.view(-1, outputs.size(-1)), tgtOutput.contiguous().view(-1))
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(decoderModel.parameters(), max_norm=1)
        optimizer.step()
        
        totalLoss += loss.item()
        

    decoderModel.eval()
    validLoss = 0
    with torch.no_grad():
        for srcSeq, tgtSeq in validLoader:
            srcSeq = srcSeq.to(device)
            tgtSeq = tgtSeq.to(device)
            
            tgtInput = tgtSeq[:, :-1]
            tgtOutput = tgtSeq[:, 1:]
            
            tgtMask = nn.Transformer.generate_square_subsequent_mask(tgtInput.size(1)).to(device)
            
            outputs = decoderModel(tgt=tgtInput, src=srcSeq, tgtMask=tgtMask)
            loss = criterion(outputs.view(-1, outputs.size(-1)), tgtOutput.contiguous().view(-1))
            validLoss += loss.item()
    
    avgTrainLoss = totalLoss / len(trainLoader)
    avgValidLoss = validLoss / len(validLoader)
    
    print(f'Epoch [{epoch+1}/{numEpochs}] Train Loss: {avgTrainLoss:.4f}, Valid Loss: {avgValidLoss:.4f}')

    if avgValidLoss < bestValidLoss:
        bestValidLoss = avgValidLoss
        patienceCounter = 0
        bestModelWeights = decoderModel.state_dict().copy()
        torch.save(decoderModel.state_dict(),"model.pth")
        print(f"Validation loss improved Saving best model")
    else:
        patienceCounter += 1
        print(f"Validation loss didn't improve. Patience: {patienceCounter}/{patience}")
        
    if patienceCounter >= patience:
        print(f"Early stopping  after {epoch+1} epochs")
        break


Epoch [1/1] Train Loss: 5.9366, Valid Loss: 4.7033
Validation loss improved Saving best model


In [21]:
# Load best model weights
if bestModelWeights is not None:
    decoderModel.load_state_dict(bestModelWeights)
    print("Loaded best model weights from early stopping")


Loaded best model weights from early stopping


In [26]:
def beamSearchDecode(model, srcInput, vocabCode, device, maxLen=500, beamWidth=5,):
    model.eval()
    
    with torch.no_grad():
        srcEmb = model.srcEmbedding(srcInput) + model.positionalEncoding[:, :srcInput.size(1), :]
        srcEmb = srcEmb.permute(1, 0, 2)  # (seq_len, 1, embed_size)
    
    beams = [{
        'sequence': [vocabCode['<sos>']],
        'score': 0.0
    }]

    for _ in range(maxLen):
        candidates = []
        for beam in beams:
            if not beam['sequence']:
                continue
            tgt = torch.tensor(beam['sequence'], device=device).unsqueeze(0)  # (1, seq_len)
            
            with torch.no_grad():
                tgtEmb = model.tgtEmbedding(tgt) + model.positionalEncoding[:, :tgt.size(1), :]
                tgtEmb = tgtEmb.permute(1, 0, 2)  # (seq_len, 1, embed_size)
                
                seq_len = tgt.size(1)
                tgtMask = nn.Transformer.generate_square_subsequent_mask(seq_len).to(device)
                
                output = model.decoder(tgtEmb, srcEmb, tgt_mask=tgtMask)
                output = output.permute(1, 0, 2)  # (1, seq_len, embed_size)
                logits = model.fc_out(output[:, -1, :])
                probs = torch.softmax(logits, dim=-1)

            topProbs, topIndices = torch.topk(probs, beamWidth * 2, dim=-1)
            for i in range(topProbs.size(-1)):
                newToken = topIndices[0][i].item()
                newScore = beam['score'] - torch.log(topProbs[0][i]).item()
                candidates.append({
                    'sequence': beam['sequence'] + [newToken],
                    'score': newScore
                })
        
        candidates = sorted(candidates, key=lambda x: x['score'])[:beamWidth]
        beams = candidates
        
        if all(beam['sequence'][-1] == vocabCode['<eos>'] for beam in beams):
            break

    bestSequence = beams[0]['sequence']
    return [vocabCode.get_itos()[idx] for idx in bestSequence 
            if idx not in {vocabCode['<sos>'], vocabCode['<eos>'],vocabCode["<unk>"]}]


In [27]:
cleanedInputs[2]


'Incorporate this change into the code: Add a csv export for caching the list of beers for display in the app\nFor your reference, this is the current state of builder.py:\n\npython\nimport ratebeer\nimport string\n\n\ndef strip_brewery_name(brewery_name, beer_name):\n    brewery_word_list = brewery_name.split()\n    for word in brewery_word_list:\n        beer_name = beer_name.replace(word, )\n    return beer_name.strip()\n\ncategories = []\ncategories.append(09)\nfor letter in string.ascii_uppercase:\n    categories.append(letter)\n\nrb = ratebeer.RateBeer()\n\nwith open(eng.user_words,w) as f:\n    for category in categories:\n        brewery_list = rb.brewers_by_alpha(category)\n        for brewery in brewery_list:\n            beer_list = brewery.get_beers()\n            for beer in beer_list:\n                index the beer name without the bewery too\n                beer_name_without_brewery = strip_brewery_name(brewery.name, beer.name)\n                f.writelines(beer_name_w

In [28]:
cleanedOutputs[1]


'```python\npython\nThis module contains a cache handler.\n\n__author__ = Aaron Steele\n\n MOL imports\nimport cache\n\n Standard Python imports\nimport json\nimport logging\nimport urllib\nimport webapp2\n\n Google App Engine imports\nfrom google.appengine.api import urlfetch\nfrom google.appengine.ext.webapp.util import run_wsgi_app\n\nclass GetHandler(webapp2.RequestHandler):\n    Request handler for cache requests.\n\n    def post(self):\n        Returns a cached value by key or None if it doesnt exist.\n        key = self.request.get(key, empty)\n        logging.info(SEARCH_KEY=s  key)\n        sql = self.request.get(sql, None)\n        cache_buster = self.request.get(cache_buster, None)\n        if not cache_buster:\n            value = cache.get(key)\n        if not value and sql:\n            url = http:mol.cartodb.comapiv2sql?s  urllib.urlencode(dict(q=sql))\n            value = urlfetch.fetch(url, deadline=60).content\n            if not json.loads(value).has_key(error) and n

In [ ]:
srcText = "can you create function to get the first element of a list"
srcTokens = [vocabText[token] for token in tokenizeText(srcText)]
srcInput = torch.tensor([vocabText['<sos>']] + srcTokens + [vocabText['<eos>']]).unsqueeze(0).to(device)

# Generate code 
generated = beamSearchDecode(
    model=decoderModel,
    srcInput=srcInput,
    vocabCode=vocabCode,
    device=device,
    beamWidth=5,
    maxLen=500,
)

print(''.join(generated))


(
